## I. Load & Process the data from Sleeper

In [1]:
from sleeper_functions import *
#Define week
week = 3
matchup_df, updated_standings, HT_Owner, HT_Score, HP_Owner, HP_Player, HP_Score = run_sleeper_weekly(week)

Loaded the League Data from Sleeper
 - Week 1 has been processed
 - Week 2 has been processed
 - Week 3 has been processed
Completed processing scores and updating standings
Saved week 3 matchup data


## II. Feed Data into GPT

In [3]:
summary = generate_summary(week, matchup_df, updated_standings)
print(summary)

Generated the summary
Hello you fantasy football savages,

We're here today to retrospect this roller-coaster of a week in our Dynasty league. The bruises are fresh, the tears are still wet, and the laughter, oh the sweet laughter rings loudly in our ears. 

First things first, let's tip our helmets to 'haamsuplo21' owned by none other than Henry, who smashed everyone with a whopping 182.8 points. Yes, you read it right! Apparently, Justin Herbert isn't the only one who can throw bombs; Henry here made sure his metaphorical football landed boom smack in the middle of our poor, unsuspecting hearts. "Stefon Diggs", who put up a handsome 15.1 pts, fast becoming the WWE superstar of his team. Bravo Henry, your fantasy prowess is as surprising as Drake’s ability to switch accents!

On the other corner, we have 'Trust the Process' by the way-downtrodden Peter - going 0-3 after a depressing 71.52 showing. Peter, my man, you gotta trust the process! That process might involve trading Trevor La

## III. Draft Summary Email via Smtpblib

In [3]:
#Load Credentials
import json
# Load the configuration from config.json
with open('config.json') as config_file:
    config_data = json.load(config_file)
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

# Load the user name and password from yaml file
user, password = config_data['gmail']['GMAIL_USER'], config_data['gmail']['GMAIL_PW']

matchup = matchup_df.to_html(index=False)
standings = updated_standings.to_html(index=False)

# Initialize the server
s = smtplib.SMTP('smtp.gmail.com', 587)
s.starttls()
s.login(user, password)

# Define message
subject = f"Dynasty FF Week {week} Recap"
summary_html = summary.replace('\n', '<br>')
body = f"""\
<html>
  <head></head>
  <body>
    <h1>{subject}</h1>
    <br>{summary_html}</br>
    <br>
    <hr>
    <p>Highest Scoring Team: {HT_Owner}: {HT_Score} points</p>
    <p>Highest Scoring Player: {HP_Owner} - {HP_Player}: {HP_Score} Points</p>
    <br>
    {standings}
  </body>
</html>
"""

recipients = list(config_data['emails']['sleeper_email_dict'].values())
sender = "peteryushunli@gmail.com"

msg = MIMEMultipart("alternative")
msg["From"] = sender
msg["To"] = ", ".join(recipients)
msg["Subject"] = subject

# Attach the HTML body
msg.attach(MIMEText(body, 'html'))

# Send email
s.sendmail(sender, recipients, msg.as_string())

# Terminate the session
s.quit()

(221,
 b'2.0.0 closing connection he21-20020a05622a601500b004120ae42859sm2647481qtb.90 - gsmtp')